# Access the camera and obtain the frame(Deprecated)

In [ ]:
import cv2
frames = []
frame_count = 0
cap = cv2.VideoCapture(0)
while True:
    ret, frame = cap.read()
    if ret:
        frames.append(frame)
        frame_count += 1

        if frame_count == 30:
            #X_data = detect_landmarks(frames)
            #inference(X_data)
            print('processing')
            frame_count = 0
            frames = []
        cv2.imshow('Camera', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()



In [ ]:

import cv2
import os

def save_frames():
    # 设置摄像头编号，如果只有一个摄像头，一般为0
    camera_id = 0

    # 设置保存图像的文件夹路径
    save_folder = 'Image_no'

    # 设置每多少帧保存一次图像
    frames_per_save = 30

    # 创建初始文件夹编号
    folder_count = 0

    # 创建初始帧计数器
    frame_count = 0

    # 检查保存文件夹是否存在，如果不存在则创建
    if not os.path.exists(save_folder):
        os.makedirs(save_folder)

    # 打开摄像头
    cap = cv2.VideoCapture(camera_id)

    while True:
        # 读取摄像头数据
        ret, frame = cap.read()

        if ret:
            # 创建新的文件夹路径
            folder_path = os.path.join(save_folder, f'folder_{folder_count}')

            # 检查文件夹是否存在，如果不存在则创建
            if not os.path.exists(folder_path):
                os.makedirs(folder_path)

            # 保存图像到指定文件夹
            image_path = os.path.join(folder_path, f'image_{frame_count % frames_per_save}.jpg')
            cv2.imwrite(image_path, frame)

            # 每30帧保存一次图像后增加文件夹编号
            if (frame_count + 1) % frames_per_save == 0:
                folder_count += 1

        # 增加帧计数器
        frame_count += 1

        # 显示图像
        cv2.imshow('Camera', frame)

        # 按下 'q' 键退出循环
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # 释放摄像头资源
    cap.release()

    # 关闭窗口
    cv2.destroyAllWindows()

# 运行保存帧的函数
save_frames()

# Load needed library

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import mediapipe as mp
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras import Sequential

In [ ]:

def detect_landmarks(image_folder):
    mp_holistic = mp.solutions.holistic.Holistic()
    image_files = sorted(os.listdir(image_folder))
    num_frames = len(image_files)
    num_landmarks = 33  # Fixed number of landmarks for pose estimation
    landmarks = np.zeros((num_frames,num_landmarks, 2))

    for i, file in enumerate(image_files):
        image_path = os.path.join(image_folder, file)
        frame = cv2.imread(image_path)

        # Convert the frame to RGB
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Detect landmarks
        results = mp_holistic.process(frame_rgb)

        if results.pose_landmarks:
            for j, landmark in enumerate(results.pose_landmarks.landmark):
                landmarks[i, j,] = [landmark.x, landmark.y]
    mp_holistic.close()
    return landmarks
def inference(X):
    CLASS_NAMES=['Left_Swipe_new','Right_Swipe_new','Stop_new','Thumbs_Down_new','Thumbs_Up_new']
    y_pred=np.argmax(model.predict(X), axis=1)
    #print(y_pred)
    print(CLASS_NAMES[y_pred[0]])
    return y_pred
def clear_files(folder_path):
    # 获取文件夹中的所有文件
    files = os.listdir(folder_path)

    # 删除每个文件
    for file in files:
        file_path = os.path.join(folder_path, file)
        if os.path.isfile(file_path):
            os.remove(file_path)
    print("-"*50)
    print("ALL Frames Cleared")

In [ ]:
save_folder = 'Image_no'
items = os.listdir(save_folder)
train_landmarks_not_move = np.empty((45,30,66))
label_not_move = []
n=0
for item in items:
    # Create the full path to the item
    if n <45:
        item_path = os.path.join(save_folder, item)
        print(f"processing {item_path}")
        train_landmarks_not_move[n] = detect_landmarks(item_path).reshape((30, 66))
        label_not_move.append(5)
        print('-'*50)
        print(f"{item_path} finished")
        print("="*50)
        n+=1
        print(n)
    
    

In [ ]:
#train_landmarks_not_move.shape
label_not_move=np.zeros((45,))
label_not_move.shape

In [ ]:
train_landmarks=np.load('train_landmarks.npy')
train_landmarks.shape

In [ ]:
label=np.ones((662,))
label.shape

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras import Sequential
from sklearn.model_selection import train_test_split

# Generate sample data
X = np.vstack((train_landmarks_not_move, train_landmarks))
y = np.concatenate((label_not_move, label))

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model2 = Sequential([
    layers.Input(shape=(30,66)),
    layers.GRU(100, return_sequences=False),
    layers.Dense(2, activation="softmax")
])
model2.compile(
    optimizer=optimizers.Adam(0.001), 
    loss="sparse_categorical_crossentropy", 
    metrics=["accuracy"]
)
filepath = "model/save"
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath, monitor='val_accuracy', save_weights_only=True, save_best_only=True, verbose=1
)
history = model2.fit(
    X_train,
    y_train,
    validation_data=(X_test, y_test),
    epochs=60,
    callbacks=[checkpoint],
)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend(['train set', 'test set'], loc='upper right')
plt.show()

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.legend(['train set', 'test set'], loc='upper right')

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
predictions = model2.predict(X_test)
y_pred = np.argmax(predictions, axis=-1)
cm=confusion_matrix(y_test,y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix - Logistic Regression')
plt.show()

In [ ]:
#model2=tf.keras.models.load_model("231012RNN_2V5.h5")
def inference2(X):
    CLASS_NAMES=['not_move','move']
    y_pred=np.argmax(model2.predict(X), axis=1)
    #print(y_pred)
    print(CLASS_NAMES[y_pred[0]])
    return y_pred

In [ ]:
model2.save("231012RNN_2V5.h5")

In [ ]:
frames = []
frame_count = 0
cap = cv2.VideoCapture(0)
#CLASS_NAMES=['Left_Swipe_new','Right_Swipe_new','Stop_new','Thumbs_Down_new','Thumbs_Up_new']
output_directory ="temp_storage"
while True:
    ret, frame = cap.read()
    if ret:
        frame_name = os.path.join(output_directory, f"frame_{frame_count}.png")
        cv2.imwrite(frame_name, frame)
        frame_count += 1

        if frame_count == 30:
            print("="*50)
            X_data = detect_landmarks(output_directory).reshape((1,30,66))
            y_pred=inference2(X_data)
            #print(f"Movement Type:{CLASS_NAMES[y_pred[0]]}, index:{y_pred[0]}")
            frame_count = 0
            clear_files(output_directory)
        cv2.imshow('Camera', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows

# framework

because the ineference process is so slow(especially with two mode), it's not working well with data collection(since they are in the same while-loop

In [2]:
import pygame
pygame.init()
def getKey(keyName):
    
    ans = False
    for eve in pygame.event.get(): pass
    keyInput = pygame.key.get_pressed()
    myKey = getattr(pygame,'K_{}'.format(keyName))
    if keyInput[myKey]:
        ans = True
    pygame.display.update()
    return ans

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import mediapipe as mp
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras import Sequential
CLASS_NAMES=['Left_Swipe_new','Right_Swipe_new','Stop_new','Thumbs_Down_new','Thumbs_Up_new']
output_directory ="temp_storage"
class model:
    def __init__(self,CLASS_NAMES,model_pass,model2,output_directory):
        self.CLASS_NAMES=CLASS_NAMES
        self.model=tf.keras.models.load_model(model_pass)
        self.model.summary()
        self.model2=tf.keras.models.load_model(model2)
        self.output_directory=output_directory
    def detect_landmarks(self,image_folder):
        mp_holistic = mp.solutions.holistic.Holistic()
        image_files = sorted(os.listdir(image_folder))
        num_frames = len(image_files)
        num_landmarks = 33  # Fixed number of landmarks for pose estimation
        landmarks = np.zeros((num_frames,num_landmarks, 2))
        for i, file in enumerate(image_files):
            image_path = os.path.join(image_folder, file)
            frame = cv2.imread(image_path)
            # Convert the frame to RGB
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            # Detect landmarks
            results = mp_holistic.process(frame_rgb)
            if results.pose_landmarks:
                for j, landmark in enumerate(results.pose_landmarks.landmark):
                    landmarks[i, j,] = [landmark.x, landmark.y]
        mp_holistic.close()
        return landmarks
    def inference(self,X):
        #CLASS_NAMES=['Left_Swipe_new','Right_Swipe_new','Stop_new','Thumbs_Down_new','Thumbs_Up_new']
        y_pred=np.argmax(self.model.predict(X), axis=1)
        #print(y_pred)
        print(self.CLASS_NAMES[y_pred[0]])
        return y_pred
    def inference2(self,X):
        CLASS_NAMES=['not_move','move']
        y_pred=np.argmax(self.model2.predict(X), axis=1)
        #print(y_pred)
        print(CLASS_NAMES[y_pred[0]])
        return y_pred
    def clear_files(self,folder_path):
        # 获取文件夹中的所有文件
        files = os.listdir(folder_path)
        # 删除每个文件
        for file in files:
            file_path = os.path.join(folder_path, file)
            if os.path.isfile(file_path):
                os.remove(file_path)
        print("-"*50)
        print("ALL Frames Cleared")
    def run(self):
        #self.clear_files(self.output_directory)
        frames = []
        frame_count = 0
        cap = cv2.VideoCapture(0)
        while True:
            ret, frame = cap.read()
            if ret:
                frame_name = os.path.join(self.output_directory, f"frame_{frame_count}.png")
                cv2.imwrite(frame_name, frame)
                frame_count += 1

                if frame_count == 30:
                    print("="*50)
                    X_data = self.detect_landmarks(self.output_directory).reshape((1,30,66))
                    self.clear_files(self.output_directory)
                    #y_check=self.inference2(X_data)
                    frame_count = 0
                    #if y_check==1:
                    y_pred=self.inference(X_data)
                    print(f"Movement Type:{self.CLASS_NAMES[y_pred[0]]}, index:{y_pred[0]}")
                        
                cv2.imshow('Camera', frame)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        cap.release()
        cv2.destroyAllWindows
def main():  
    model(CLASS_NAMES,"231008RNNV5.h5","231012RNN_2V5.h5",output_directory).run()
if __name__ == '__main__':
    main()

In [1]:
clear_files(output_directory)

NameError: name 'clear_files' is not defined

In [ ]:
!pip install djitellopy

In [ ]:
!pip install pygame